## Overview: Multiclass classification
- A classification task with more than two classes; e.g., classify a set of images of fruits which may be oranges, apples, or pears. Multiclass classification assumes each sample is assigned to one and only one label (mutually exclusive).
- Common examples include image classification (cat, dog, human, etc.) or handwritten digit recognition (classifying an image into a digit from 0 to 9).
- In machine learning, multiclass (multinomial) classification is the problem of classifying instances into one of three or more classes (two classes is binary classification).
- Multiclass classification should not be confused with multi-label classification, where multiple labels can be predicted for each instance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib as mpl
from sklearn.datasets import make_blobs

In [ ]:
# some helper function below to plot data
def plt_mc_data(ax, X, y, classes,  class_labels=None, map=plt.cm.Paired, 
                legend=False, size=50, m='o', equal_xy = False):
    """ Plot multiclass data. Note, if equal_xy is True, setting ylim on the plot may not work """
    
    for i in range(classes):
        idx = np.where(y == i)
        col = len(idx[0])*[i]
        label = class_labels[i] if class_labels else "c{}".format(i)
        # Use numeric class labels with a colormap to avoid warnings about vmin/vmax being ignored
        ax.scatter(X[idx, 0], X[idx, 1],  marker=m,
                    c=col, cmap=map, vmin=0, vmax=classes-1,
                    s=size, label=label)
    if legend: ax.legend()
    if equal_xy: ax.axis("equal")

def plt_mc(X_train,y_train,classes, centers, std):
    css = np.unique(y_train)
    fig,ax = plt.subplots(1,1,figsize=(3,3))
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    plt_mc_data(ax, X_train,y_train,classes, map=plt.cm.Paired, legend=True, size=50, equal_xy = False)
    ax.set_title("Multiclass Data")
    ax.set_xlabel("x0")
    ax.set_ylabel("x1")
    plt.show()
    

## 1. Prepare and visualize our data
We will use Scikit-Learn `make_blobs` function to make a training data set with 4 categories as shown in the plot below.

In [ ]:
# make 4-class dataset for classification
classes = 4
m = 500
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X, y = make_blobs(n_samples=m, centers=centers, cluster_std=std,random_state=30)

# add more noise to the data
np.random.seed(42)
X += np.random.normal(0, 0.75, X.shape)


In [ ]:
plt_mc(X,y,classes, centers, std=std)

In [ ]:
# split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# plot side by side subplots on training and test data
fig,ax = plt.subplots(1,2,figsize=(6,3))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
plt_mc_data(ax[0], X_train,y_train,classes, map=plt.cm.Paired, legend=True, size=50, equal_xy = False)
ax[0].set_title("Training Data")
ax[0].set_xlabel("x0")
ax[0].set_ylabel("x1")
plt_mc_data(ax[1], X_test,y_test,classes, map=plt.cm.Paired, legend=True, size=50, equal_xy = False)
ax[1].set_title("Test Data")
ax[1].set_xlabel("x0")
ax[1].set_ylabel("x1")
plt.show()


##  Softmax (Multinominal Logistic Regression)

###  Softmax Function

In softmax regression, N outputs are generated and one output is selected as the predicted category. In this case a vector $\mathbf{z}$ is generated by a linear function which is applied to a softmax function. The softmax function converts $\mathbf{z}$  into a probability distribution as described below. After applying softmax, each output will be between 0 and 1 and the outputs will add to 1, so that they can be interpreted as probabilities. The larger inputs  will correspond to larger output probabilities.
<center>  <img  src="./images/C2_W2_SoftmaxReg_NN.png" width="600" />  

The softmax function can be written:

$$a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} \tag{1}$$


Which shows the output is a vector of probabilities. The first entry is the probability the input is the first category given the input $\mathbf{x}$ and parameters $\mathbf{w}$ and $\mathbf{b}$.  
Let's create a NumPy implementation:



In [ ]:
def my_softmax(z):
    ez = np.exp(z)              #element-wise exponenial
    sm = ez/np.sum(ez)
    return(sm)

###  Softmax Cost
<center> <img  src="./images/C2_W2_SoftMaxCost.png" width="400" />    <center/>

The loss function associated with Softmax, the cross-entropy loss, is:
\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{if $y=N$}
  \end{cases} \tag{3}
\end{equation}

Where y is the target category for this example and $\mathbf{a}$ is the output of a softmax function. In particular, the values in $\mathbf{a}$ are probabilities that sum to one.
>**Recall:** In this course, Loss is for one example while Cost covers all examples. 
 
 
Note in (3) above, only the line that corresponds to the target contributes to the loss, other lines are zero. To write the cost equation we need an 'indicator function' that will be 1 when the index matches the target and zero otherwise. 
    $$\mathbf{1}\{y == n\} = =\begin{cases}
    1, & \text{if $y==n$}.\\
    0, & \text{otherwise}.
  \end{cases}$$
Now the cost is:
\begin{align}
J(\mathbf{w},b) = -\frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{N}  1\left\{y^{(i)} == j\right\} \log \frac{e^{z^{(i)}_j}}{\sum_{k=1}^N e^{z^{(i)}_k} }\right] \tag{4}
\end{align}

Where $m$ is the number of examples, $N$ is the number of outputs. This is the average of all the losses.



###  Multinominal Logistic Regression

Let's build a model

In [ ]:
from sklearn.linear_model import LogisticRegression


# Train multinomial logistic regression classifier
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, y_train)

# Get the predicted probabilities for each class
y_pred_probabilities = logreg_classifier.predict_proba(X_test)
y_pred_probabilities[:5]

In [ ]:
y_pred = logreg_classifier.predict(X_test)
y_pred

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix
pd.DataFrame(confusion_matrix(y_test,y_pred))
print(round(accuracy_score(y_test, y_pred), 2))

In [ ]:
confusion_matrix(y_test,y_pred)

###  Softmax Classifier in Neural Nets


#### MLPClassifier from Sklearn


`MLPClassifier` in scikit-learn is a multi-layer perceptron (MLP) classifier, which is a type of artificial neural network. It can be used for multi-class classification tasks by learning a mapping from input features to output class labels.

In [ ]:
# build a neural network classifier on the data
from sklearn.neural_network import MLPClassifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
mlp_classifier.fit(X_train, y_train)

# make prediction on the test data
y_pred = mlp_classifier.predict(X_test)

# output the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# output the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}".format(accuracy))

In [ ]:
y_pred_prob = mlp_classifier.predict_proba(X_test)
print("Prediction Probabilities:", y_pred_prob[:5])

In [ ]:
# add all probabities for each instance 
y_pred_prob.sum(axis=1)

Noticeably, the sum of probabilities for each instance totals to 1, a characteristic indicative of the use of softmax for multi-class classification. Now, let's take a step further and explicitly construct a neural network classifier, specifying the output layer with softmax activation

####  Keras

In [ ]:
# Set PyTorch as the backend for Keras before importing Keras
import os
os.environ["KERAS_BACKEND"] = "torch"

# Now import Keras (it will use PyTorch as backend)
import keras
from keras.models import Sequential
from keras.layers import Dense

# build a neural network classifier on the data
keras_classifier = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)

keras_classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
)

keras_classifier.fit(
    X_train, y_train,
    epochs=10
)

Because the softmax is integrated into the output layer, the output is a vector of probabilities.

In [ ]:
# show first few predicted probability vectors from the Keras (torch-backend) model
p_nonpreferred = keras_classifier.predict(X_test)
print(p_nonpreferred[:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

In [ ]:
# output accuracy and confusion matrix
y_pred = np.argmax(p_nonpreferred, axis=1)
print(round(accuracy_score(y_test, y_pred), 2))
print(confusion_matrix(y_test, y_pred))


##  Accuracy, precision, recall, and f1 score for multiclass classification

In multiclass classification, accuracy is calculated in the same way as in binary classification, but precision, recall, and F1-score are calculated for each class individually.

Please refer to the following figure for the caculation

<img src="https://www.sefidian.com/wp-content/uploads/2023/04/image-40.png" alt="Image" style="width:450px;"/>

In [ ]:
## output the performance metrics
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In multiclass classification, these metrics are calculated separately for each class, and then typically aggregated using some form of averaging (e.g., micro-average, macro-average, weighted average) to obtain overall performance measures. This allows us to evaluate the classifier's performance on each class individually as well as on the entire dataset. 

In [ ]:
## output the precision, recall and f1 score
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision: {:.2f}".format(precision_score(y_test, y_pred, average='macro')))
print("Recall: {:.2f}".format(recall_score(y_test, y_pred, average='macro')))
print("F1 Score: {:.2f}".format(f1_score(y_test, y_pred, average='macro')))


In multiclass classification, macro-average and weighted average are two commonly used methods for aggregating performance metrics (such as precision, recall, and F1-score) across multiple classes to obtain overall performance measures.

Below is an explanation of each:


### Macro-averaging
**verage the precision and recall across all classes** to get the final macro-averaged precision and recall scores.

<img src="https://assets-global.website-files.com/6266b596eef18c1931f938f9/644afed8b72fe836c6eae060_class_guide_multi_abc13.png" alt="Image" style="width:450px;"/>


In [ ]:
## output the precision, recall and f1 score
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision: {:.2f}".format(precision_score(y_test, y_pred, average='weighted')))
print("Recall: {:.2f}".format(recall_score(y_test, y_pred, average='weighted')))
print("F1 Score: {:.2f}".format(f1_score(y_test, y_pred, average='weighted')))

### Weighted averaging
This approach takes into account the balance of classes. You weigh each class based on its representation in the dataset. Then, you compute precision and recall as a weighted average of the precision and recall in individual classes.

Simply put, it would work like macro-averaging, but instead of dividing precision and recall by the number of classes, you give each class a fair representation based on the proportion it takes in the dataset. 

This approach is useful if you have an imbalanced dataset but want to assign larger importance to classes with more examples. 

## Congratulations!
In this lab you 
* Extended the binary classification to multiclass classification
* Explored different ML classifiers for multiclass classification
* Got to know softmax function, which is widely used in neural networks
* Explored the distinctions in performance metrics between multiclass and binary classification.
